### 專案名稱: 網頁爬蟲練習
### 功能描述: 抓取證券商承銷有價證券公告等相關資料
### [證券商承銷有價證券公告](http://web.csa.org.tw/Edoc2/Default.aspx "抓取證券商承銷有價證券公告")
### [證券商承銷有價證券公告_限專業投資人投資債券承銷公告](http://web2.twsa.org.tw/Bond/Home/Index "證券商承銷有價證券公告_限專業投資人投資債券承銷公告")
### [悠債網發行公告](http://www.yobond.com.tw/Form/pub/ "悠債網發行公告")
### 版權所有: Dunk  
### 程式撰寫: Dunk  
### 撰寫日期：2020/04/16
### 改版日期:  
### 改版備註:

#### 呼叫所需套件

In [111]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

#### 抓取資料
參考網址

[抓取所有指定類別項目](https://stackoverflow.com/questions/43261897/retrieving-a-subset-of-hrefs-from-findall-in-beautifulsoup)

[刪除dataframe row](https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/ '刪除dataframe row')

In [112]:
# 指定網址:證券商承銷有價證券公告
consdb_url = "http://web.csa.org.tw/Edoc2/Default.aspx"

consdb_content = requests.post(consdb_url)
soup = BeautifulSoup(consdb_content.content, 'html.parser')

# 抓取下拉清單
pattern = re.compile("ctl00_cphMain_rblReportType_[0-9]+")
name_array = []
value_array = []

for link in soup.find_all('input'):
    if pattern.match(link['id']):
        value_array.append(link['value'])
        
for link in soup.find_all('label'):
    if pattern.match(link['for']):
        name_array.append(link.text)
        
lookItme = pd.DataFrame({'key':name_array,'value':value_array})
lookItme = lookItme[lookItme.key != '限專業投資人投資債券承銷公告']

year = input('請輸入查詢年度(西元年,yyyy):')
for i in range(lookItme.shape[0]):
    #各類報表顯示
    print('['+lookItme.iloc[i]['key']+']')
    select_value = lookItme.iloc[i]['value']
    payload = {
        '__EVENTARGUMENT': '',
        '__EVENTTARGET': 'ctl00$cphMain$rblReportType$'+str(i),
        '__EVENTVALIDATION': soup.find('input', attrs={'name': '__EVENTVALIDATION'})['value'],
        '__LASTFOCUS': '',
        '__VIEWSTATE': soup.find('input', attrs={'name': '__VIEWSTATE'})['value'],    
        '__VIEWSTATEGENERATOR': soup.find('input', attrs={'name': '__VIEWSTATEGENERATOR'})['value'],       
        'ctl00$cphMain$ddlYear': year,
        'ctl00$cphMain$rblReportType': select_value
    }    
    post = requests.post(consdb_url, data=payload)    
    df_original = pd.DataFrame(pd.read_html(post.text, encoding='utf-8')[6])        
    print(df_original.head())
    

# 指定網址:證券商承銷有價證券公告(限專業投資人投資債券承銷公告)
chi_year = int(year)-1911
consdb_url = "http://web2.twsa.org.tw/Bond/Home/Index?y=" + str(chi_year)
print('[限專業投資人投資債券承銷公告]')
df_original = pd.DataFrame(pd.read_html(consdb_url, encoding='utf-8')[0])
print(df_original.head())

請輸入查詢年度(西元年,yyyy):2018
[承銷公告]
       序號        申報日期           主辦承銷商                          案件名稱  方式  發行性質  \
0  107001  2017/12/27      兆豐證券股份有限公司                    台境企業股份有限公司  包銷   公司債   
1  107002  2017/12/28      元大證券股份有限公司                        貿聯控股公司  包銷  現金增資   
2  107003  2017/12/28      元大證券股份有限公司  台北金融大樓股份有限公司106年度第一期無擔保普通公司債  包銷   公司債   
3  107004  2017/12/28  華南永昌綜合證券股份有限公司  華南金如融控股份有限公司106年度第1期無擔保普通公司債  包銷   公司債   
4  107005  2017/12/29      兆豐證券股份有限公司                    裕融企業股份有限公司  包銷   公司債   

              發行種類   配售方式一 配售方式二 案件狀態  公告檔  
0         有擔保轉換公司債    詢價圈購   NaN   正常  NaN  
1              普通股  公開申購配售   NaN   正常  NaN  
2  普通公司債(限售予專業投資人)    洽商銷售   NaN   正常  NaN  
3  普通公司債(限售予專業投資人)    洽商銷售   NaN   正常  NaN  
4  普通公司債(限售予專業投資人)    洽商銷售   NaN   正常  NaN  
[競拍公告/開標統計表]
       序號             發行公司         主辦承銷商  發行性質  承銷股數(千股/張)  競拍股數(千股/張)  \
0  107001  聯廣傳播股份有限公司(未開標)  國泰綜合證券股份有限公司  初次上市        6727        4680   
1  107002     金穎生物科技股份有限公司  富邦綜合證券股份有限公司  初次上櫃      

In [113]:
# 指定網址:悠債網發行公告
consdb_url = "http://www.yobond.com.tw/Form/pub/"
print('[悠債網]')
df_original = pd.DataFrame(pd.read_html(consdb_url, encoding='utf-8')[3])
print(df_original.head())

[悠債網]
   發行機構代碼 可轉債名稱  發行時轉換價 擔保有無        掛牌日期 公開說明書
0    6532   瑞耘一      44    有  2020-03-18    下載
1    2228   劍麟二     106    無  2020-03-09    下載
2    8349   恒耀四      62    無  2020-03-06    下載
3    6441   廣錠一      62    有  2020-03-05    下載
4    6441   廣錠二      62    有  2020-03-05    下載
